In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm

# Dados NSL-KDD

In [2]:
kddtrain = pd.read_csv("kddtrain_onehot.csv").to_numpy()
kddtest = pd.read_csv("kddtest_onehot.csv").to_numpy()

# Abordagem Autoencoder-KNN

In [3]:
data = np.concatenate((kddtrain,kddtest), axis=0)
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

acc_per_fold = []
loss_per_fold = []
num_folds = 10

kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas KMeans
matrix_kmeans = np.zeros((num_folds,2,2))
accuracy_kmeans = np.zeros(num_folds)
recall_kmeans = np.zeros(num_folds)
precision_kmeans = np.zeros(num_folds)
f1_kmeans = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data[::, 0:data.shape[1]-3], data[::, data.shape[1]-1]):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTest = data[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain[:,Train-1] == 1)
    anTrain = np.count_nonzero(dataTrain[:, Train-1] == 0)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain[:,Train-1] == 1)]
    anomalyTrain = dataTrain[np.where(dataTrain[:,Train-1] == 0)]
    
    #Divide dados para treinamento
    porcen = 0.50
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, normalTrain.shape[0]),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anomalyTrain.shape[0]),j))
    numbers_autoencoder_normal = np.array(list(set(np.arange(0, normalTrain.shape[0])) - set(numbers_knn_normal)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    
    #Dados Autoencoder
    autoencoderTrain = normalTrain[numbers_autoencoder_normal]
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(autoencoderTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)

    scaler1 = MinMaxScaler()
    scaler1.fit(knnTrain)
    knnTrain = scaler1.transform(knnTrain)

    dataTest = scaler1.transform(dataTest)
    
    ###Separação das Características e classes
    autoencoderTrain_class = autoencoderTrain[::, autoencoderTrain.shape[1]-2:autoencoderTrain.shape[1]]
    autoencoderTrain = autoencoderTrain[::, 0:autoencoderTrain.shape[1]-3]

    knnTrain_class = knnTrain[::, knnTrain.shape[1]-2:knnTrain.shape[1]]
    knnTrain = knnTrain[::, 0:knnTrain.shape[1]-3]

    modelTest_class = dataTest[::, dataTest.shape[1]-2:dataTest.shape[1]]
    modelTest = dataTest[::, 0:dataTest.shape[1]-3]
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(32, activation=activation)(input_vector)
    en2 = layers.Dense(16, activation=activation)(en1)
    de1 = layers.Dense(32, activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)
    
    encoder = keras.Model(input_vector, en2)
    encoded_input = keras.Input(shape=(16,))
    decoder_layer = autoencoder.layers[-2]
    decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    
    #####################################################
    ####Erro de Reconstrução
    #Teste
    pred_ErroRecons = autoencoder.predict(modelTest)
    
    #Extração do erro
    mse_ErroRecons = np.mean(np.power(modelTest - pred_ErroRecons, 2), axis=1)
    error_df_ErroRecons = pd.DataFrame({'reconstruction_error': mse_ErroRecons,
                              'true_class': modelTest_class[:,1]})
    
    #Limiar de Separação
    media = np.mean(error_df_ErroRecons['reconstruction_error'])
    desv = np.std(error_df_ErroRecons['reconstruction_error'])
    Z = 0
    teta = media + Z*desv
    
    #Separação dos erros
    predi_ErroRecons = np.zeros(modelTest.shape[0])
    error_ErroRecons = error_df_ErroRecons.to_numpy()
    predi_ErroRecons[np.where(error_ErroRecons[:,0] <= teta)] = 1
    
    #Métricas
    accuracy_ErroRecons[count] = accuracy_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    recall_ErroRecons[count] = recall_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    precision_ErroRecons[count] = precision_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    f1_ErroRecons[count] = f1_score(error_ErroRecons[:,1], predi_ErroRecons[:])
    matrix_ErroRecons[count] = confusion_matrix(error_ErroRecons[:,1], predi_ErroRecons[:])
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class[::,1]})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_knn, axis=1)
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_encoder[:,0:error_encoder.shape[1] - 2], error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class[::,1]})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer, error_knn, axis=1)
    predict_knn = neigh.predict(error_encoder[:,0:error_encoder.shape[1] - 2])
    
    #Métricas
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn[count] = confusion_matrix(error_knn[:,1], predict_knn[:])
    
    #####################################################
    ####KMeans
    ###Treinamento KMeans 
    pred_kmeans = autoencoder.predict(knnTrain)
    mse_kmeans = np.mean(np.power(knnTrain - pred_kmeans, 2), axis=1)
    error_df_kmeans = pd.DataFrame({'reconstruction_error': mse_kmeans,
                                     'true_class': knnTrain_class[::,1]})
    error_kmeans = error_df_kmeans.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_kmeans, axis=1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(error_encoder[:,0:error_encoder.shape[1]-2])
    kmeans.labels_

    #Teste
    pred_kmeans = autoencoder.predict(modelTest)
    mse_kmeans = np.mean(np.power(modelTest - pred_kmeans, 2), axis=1)
    error_df_kmeans = pd.DataFrame({'reconstruction_error': mse_kmeans,
                                     'true_class': modelTest_class[::,1]})
    error_kmeans = error_df_kmeans.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_kmeans, axis=1)
    predict_kmeans = kmeans.predict(error_encoder[:,0:error_encoder.shape[1]-2])

    #Métricas
    accuracy_kmeans[count] = accuracy_score(error_kmeans[:,1], predict_kmeans[:])
    recall_kmeans[count] = recall_score(error_kmeans[:,1], predict_kmeans[:])
    precision_kmeans[count] = precision_score(error_kmeans[:,1], predict_kmeans[:])
    f1_kmeans[count] = f1_score(error_kmeans[:,1], predict_kmeans[:])
    matrix_kmeans[count] = confusion_matrix(error_kmeans[:,1], predict_kmeans[:])
    
    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': knnTrain_class[::,1]})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_svm, axis=1)
    clf = svm.SVC(kernel='rbf')
    clf.fit(error_encoder[:,0:error_encoder.shape[1] - 2], error_svm[:,1])
    
    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': modelTest_class[::,1]})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_svm, axis=1)
    predict_svm = clf.predict(error_encoder[:,0:error_encoder.shape[1]-2])
    
    #Métricas
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm[count] = confusion_matrix(error_svm[:,1], predict_svm[:])

    count = count + 1
    fold_no = fold_no + 1

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 1s 2ms/step - loss: 0.1141
Epoch 2/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0929
Epoch 3/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0704
Epoch 4/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0597
Epoch 5/50
136/136 [==============================] - 

136/136 [==============================] - 0s 2ms/step - loss: 0.0259
Epoch 6/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0227
Epoch 7/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0217
Epoch 8/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0209
Epoch 9/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0204
Epoch 10/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0199
Epoch 11/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0196
Epoch 12/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0193
Epoch 13/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0191
Epoch 14/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0189
Epoch 15/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0188
Epoch 16/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0187
Epoch 17/50
136/136 [===========

Epoch 21/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0183
Epoch 22/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0183
Epoch 23/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0182
Epoch 24/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0182
Epoch 25/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0181
Epoch 26/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0181
Epoch 27/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0181
Epoch 28/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0180
Epoch 29/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 30/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 31/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 32/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0179
Epoch 33/50
136/

136/136 [==============================] - 0s 2ms/step - loss: 0.0156
Epoch 38/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0156A: 0s - loss: 0.01
Epoch 39/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 40/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 41/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 42/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 43/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 44/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 45/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 46/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 47/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 48/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 49/5

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/50
Please report this to the TensorFlow team. When filing t

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 4s 1ms/step - loss: 0.0521
Epoch 2/50
136/136 [==============================] - 0s 1ms/step - loss: 0.0190
Epoch 3/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0103
Epoch 4/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0088
Epoch 5/50
136/136 [==============================] - 

136/136 [==============================] - 0s 2ms/step - loss: 0.0234
Epoch 6/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0225
Epoch 7/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0220
Epoch 8/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0217
Epoch 9/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0214
Epoch 10/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0212A: 0s - loss: 0.02
Epoch 11/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0211
Epoch 12/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0209
Epoch 13/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0208
Epoch 14/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0207
Epoch 15/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0206
Epoch 16/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0205
Epoch 17/50
13

136/136 [==============================] - 0s 2ms/step - loss: 0.0054
Epoch 22/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0053
Epoch 23/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0053
Epoch 24/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0053
Epoch 25/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0053
Epoch 26/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 27/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 28/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 29/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 30/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 31/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0051
Epoch 32/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0051
Epoch 33/50
136/136 [=======

136/136 [==============================] - 0s 2ms/step - loss: 0.0046
Epoch 38/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0046
Epoch 39/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0046
Epoch 40/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 41/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 42/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 43/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 44/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 45/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 46/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 47/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 48/50
136/136 [==============================] - 0s 2ms/step - loss: 0.0045
Epoch 49/50
136/136 [=======

In [4]:
print("Autoencoder - Erro de Reconstrução:\n", np.mean(matrix_ErroRecons, axis=0), "\nAcurácia: ", np.mean(accuracy_ErroRecons),' +/- ',np.std(accuracy_ErroRecons), "\nRecall: ", np.mean(recall_ErroRecons),' +/- ',np.std(recall_ErroRecons), "\nPrecision: ", np.mean(precision_ErroRecons),' +/- ',np.std(precision_ErroRecons), "\nF1: ", np.mean(f1_ErroRecons),' +/- ',np.std(f1_ErroRecons))

print("\nAutoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - KMeans:\n", np.mean(matrix_kmeans, axis=0), "\nAcurácia: ", np.mean(accuracy_kmeans), ' +/- ', np.std(accuracy_kmeans), "\nRecall: ", np.mean(recall_kmeans), ' +/- ', np.std(recall_kmeans), "\nPrecision: ", np.mean(precision_kmeans), ' +/- ', np.std(precision_kmeans), "\nF1: ", np.mean(f1_kmeans), ' +/- ', np.std(f1_kmeans))

Autoencoder - Erro de Reconstrução:
 [[5251.9 1894.3]
 [ 267.7 7437.6]] 
Acurácia:  0.8544259035569828  +/-  0.04501568343333686 
Recall:  0.9652776196827547  +/-  0.02077381645018394 
Precision:  0.802572675625022  +/-  0.0627527792549623 
F1:  0.8746043495803487  +/-  0.033239947036956874

Autoencoder - KNN:
 [[7002.6  143.6]
 [ 149.5 7555.8]] 
Acurácia:  0.9802645990843637  +/-  0.0022597227565053883 
Recall:  0.9805966952994375  +/-  0.0026964649947879265 
Precision:  0.9813484003959501  +/-  0.002904078374304816 
F1:  0.9809693586247359  +/-  0.002208317714805286

Autoencoder - SVM:
 [[6707.9  438.3]
 [ 186.8 7518.5]] 
Acurácia:  0.9579099552003806  +/-  0.002182149415568962 
Recall:  0.9757495810833985  +/-  0.0029938986531610453 
Precision:  0.9449384294850202  +/-  0.004296506917397142 
F1:  0.9600867736784311  +/-  0.0020736718338015517

Autoencoder - KMeans:
 [[3801.2 3345. ]
 [3308.4 4396.9]] 
Acurácia:  0.5520084754091015  +/-  0.3000568046022361 
Recall:  0.570414040070246

In [5]:
#Salva Resultados
with open('Resultados_Erro+InterLayer.csv', 'a', newline='') as csvfile:
    fieldnames = ['Algoritmo','Métrica', 'Média', 'Desvio']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    writer.writerow({'Algoritmo': 'ErroRecons_acurácia','Métrica': accuracy_ErroRecons, 'Média': np.mean(accuracy_ErroRecons), 'Desvio': np.std(accuracy_ErroRecons)})
    writer.writerow({'Algoritmo': 'ErroRecons_recall','Métrica': recall_ErroRecons, 'Média': np.mean(recall_ErroRecons), 'Desvio': np.std(recall_ErroRecons)})
    writer.writerow({'Algoritmo': 'ErroRecons_precision','Métrica': precision_ErroRecons, 'Média': np.mean(precision_ErroRecons), 'Desvio': np.std(precision_ErroRecons)})
    writer.writerow({'Algoritmo': 'ErroRecons_f1','Métrica': f1_ErroRecons, 'Média': np.mean(f1_ErroRecons), 'Desvio': np.std(f1_ErroRecons)})
    writer.writerow({'Algoritmo': 'ErroRecons_matrix','Métrica': matrix_ErroRecons, 'Média': np.mean(matrix_ErroRecons, axis=0), 'Desvio': np.std(matrix_ErroRecons, axis=0)})
    
    writer.writerow({'Algoritmo': 'KNN_acurácia','Métrica': accuracy_knn, 'Média': np.mean(accuracy_knn), 'Desvio': np.std(accuracy_knn)})
    writer.writerow({'Algoritmo': 'KNN_recall','Métrica': recall_knn, 'Média': np.mean(recall_knn), 'Desvio': np.std(recall_knn)})
    writer.writerow({'Algoritmo': 'KNN_precision','Métrica': precision_knn, 'Média': np.mean(precision_knn), 'Desvio': np.std(precision_knn)})
    writer.writerow({'Algoritmo': 'KNN_f1','Métrica': f1_knn, 'Média': np.mean(f1_knn), 'Desvio': np.std(f1_knn)})
    writer.writerow({'Algoritmo': 'KNN_matrix','Métrica': matrix_knn, 'Média': np.mean(matrix_knn, axis=0), 'Desvio': np.std(matrix_knn, axis=0)})
    
    writer.writerow({'Algoritmo': 'SVM_acurácia','Métrica': accuracy_svm, 'Média': np.mean(accuracy_svm), 'Desvio': np.std(accuracy_svm)})
    writer.writerow({'Algoritmo': 'SVM_recall','Métrica': recall_svm, 'Média': np.mean(recall_svm), 'Desvio': np.std(recall_svm)})
    writer.writerow({'Algoritmo': 'SVM_precision','Métrica': precision_svm, 'Média': np.mean(precision_svm), 'Desvio': np.std(precision_svm)})
    writer.writerow({'Algoritmo': 'SVM_f1','Métrica': f1_svm, 'Média': np.mean(f1_svm), 'Desvio': np.std(f1_svm)})
    writer.writerow({'Algoritmo': 'SVM_matrix','Métrica': matrix_svm, 'Média': np.mean(matrix_svm, axis=0), 'Desvio': np.std(matrix_svm, axis=0)})
    
    writer.writerow({'Algoritmo': 'KMeans_acurácia','Métrica': accuracy_kmeans, 'Média': np.mean(accuracy_kmeans), 'Desvio': np.std(accuracy_kmeans)})
    writer.writerow({'Algoritmo': 'KMeans_recall','Métrica': recall_kmeans, 'Média': np.mean(recall_kmeans), 'Desvio': np.std(recall_kmeans)})
    writer.writerow({'Algoritmo': 'KMeans_precision','Métrica': precision_kmeans, 'Média': np.mean(precision_kmeans), 'Desvio': np.std(precision_kmeans)})
    writer.writerow({'Algoritmo': 'KMeans_f1','Métrica': f1_kmeans, 'Média': np.mean(f1_kmeans), 'Desvio': np.std(f1_kmeans)})
    writer.writerow({'Algoritmo': 'KMeans_matrix','Métrica': matrix_kmeans, 'Média': np.mean(matrix_kmeans, axis=0), 'Desvio': np.std(matrix_kmeans, axis=0)})